In [1]:
import sqlite3
conn = sqlite3.connect('definitions.db')
cur = conn.cursor()

In [2]:
import tkinter as tk
class SearchSystem(tk.Tk):
    def __init__(self):
        super().__init__()
        
        self.result = [] #
        self.definition_id = 0
        
        self.term_text_frame = tk.LabelFrame(self, text = 'Введите термин')
        self.term_text = tk.Text(self.term_text_frame, width = 40, height = 2)
        self.term_text.pack()
        self.term_text_frame.pack(side = tk.TOP)

        self.definition_frame = tk.LabelFrame(self, text = 'Источник определения')
        self.definition = tk.Message(self.definition_frame, width = 600)
        self.definition.config(text = 'Текст определения')
        self.definition.pack()
        self.definition_frame.pack(side = tk.TOP)

        tk.Button(self, text = '-->', command = self.next).pack()

        self.label = tk.Label(self, text = '')
        self.label.pack()

        self.find_frame = tk.Frame(self)
        self.find_btn = tk.Button(self.find_frame, text = 'Найти определение', width = 15, height = 2, command = self.save_term)
        self.find_btn.pack(side = tk.RIGHT)
        self.find_frame.pack()

    def next(self):
        if self.result != []:
            self.definition_id += 1
            if self.definition_id > len(self.result):
                self.definition_id -= len(self.result)
            self.definition.config(text = self.result[self.definition_id - 1][0])
            self.definition_frame.config(text = self.result[self.definition_id - 1][1])
            self.label.config(text = str(self.definition_id) + ' из ' + str(len(self.result)))

    def save_term(self):
        term_query = self.term_text.get('1.0', tk.END)[:-1]
        if term_query != '':
            sql_query = "SELECT definition, document FROM terms WHERE firstterm = ? OR secondterm = ?;"
            res = cur.execute(sql_query, (term_query, term_query))
            
            self.result = [(definition, document) for definition, document in res.fetchall()]

            if self.result == []:
                self.definition.config(text = 'Такого термина нет в базе данных')
                self.label.config(text = '')
                self.definition_frame.config(text = 'Нет такого источника')
                self.find_btn.config(background = 'white')
            else:
                self.definition_id = 1
                self.label.config(text = str(self.definition_id) + ' из ' + str(len(self.result)))

                self.definition.config(text = self.result[self.definition_id - 1][0])
                self.definition_frame.config(text = self.result[self.definition_id - 1][1])

                self.find_btn.config(background = 'green')
            
    

application = SearchSystem()
application.title('Поисковая система')
application.geometry('880x650')
application.mainloop()